In [1]:
import pandas as pd
import re
import chardet

In [2]:
# Load the dataset
df = pd.read_excel("netflix_titles.xlsx")  

# Number of rows and columns
num_rows, num_cols = df.shape
print(f"\n Number of Rows: {num_rows}")
print(f" Number of Columns: {num_cols}")

# Column headers
print("\n Column Headers:")
print(df.columns.tolist())

# Data types of each column
print("\n Data Types:")
print(df.dtypes)

# Missing values count and percentage
print("\n Missing Values Analysis:")
missing_counts = df.isnull().sum()
missing_percentages = (missing_counts / num_rows * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_percentages
})
print(missing_df[missing_df['Missing Count'] > 0])



 Number of Rows: 8807
 Number of Columns: 12

 Column Headers:
['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']

 Data Types:
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

 Missing Values Analysis:
            Missing Count  Missing %
director             2634      29.91
cast                  825       9.37
country               831       9.44
date_added             10       0.11
rating                  7       0.08


In [3]:
def fix_mojibake(text):
    if pd.isnull(text):
        return text
    if not isinstance(text, str):
        return text

    try:
        # Encode as bytes (assume current is mis-decoded Latin-1)
        raw_bytes = text.encode('latin1', errors='ignore')
        # Decode properly to UTF-8
        fixed = raw_bytes.decode('utf-8', errors='ignore')
        return fixed
    except Exception:
        return text

def clean_text(val):
    if pd.isnull(val):
        return val
    if not isinstance(val, str):
        val = str(val)

    val = fix_mojibake(val)

    # Normalize text and remove invisible/extra characters
    val = re.sub(r'[\r\n\t\xa0]', ' ', val)  # remove newlines, tabs, non-breaking spaces
    val = re.sub(' +', ' ', val).strip()     # remove extra spaces

    return val

# Load Excel file
df = pd.read_excel("netflix_titles.xlsx")

# Clean all object-type columns
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].apply(clean_text)



In [4]:
# Save cleaned dataset for inspection
df.to_excel("netflix_clean.xlsx", index=False)
print("saved")

saved


In [5]:
# Remove duplicate rows based on the "title" column
df_deduplicated = df.drop_duplicates(subset=["title", "release_year"])

In [6]:
# Total original rows
original_count = len(df)

# Deduplicated rows
deduplicated_count = len(df_deduplicated)

# Number of duplicate rows removed
duplicates_removed = original_count - deduplicated_count

print(f"Rows after removing duplicates: {deduplicated_count}")
print(f"Number of duplicate rows removed: {duplicates_removed}")

Rows after removing duplicates: 8803
Number of duplicate rows removed: 4


In [7]:
df_clean = df_deduplicated

In [8]:
# Missing values count and percentage
print("\nMissing Values Analysis:")
num_rows = len(df_clean)
missing_counts = df_clean.isnull().sum()
missing_percentages = (missing_counts / num_rows * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_percentages
})
print(missing_df[missing_df['Missing Count'] > 0])


Missing Values Analysis:
            Missing Count  Missing %
director             2633      29.91
cast                  825       9.37
country               830       9.43
date_added             10       0.11
rating                  7       0.08


In [10]:
df_clean = df_clean.copy()

# Fill missing values safely
df_clean.loc[:, ['director', 'cast', 'country', 'rating']] = df_clean[['director', 'cast', 'country', 'rating']].fillna('Unknown')

# Save cleaned data
df_clean.to_excel("netflix_fully_cleaned.xlsx", index=False)
print("saved!")

saved!


In [11]:
# Explode Genre (listed_in) for power bi model
df_clean['listed_in'] = df_clean['listed_in'].fillna('').apply(lambda x: [i.strip() for i in x.split(',') if i.strip()])
genre_df = df_clean.explode('listed_in')

#  Explode Cast for power bi model
df_clean['cast'] = df_clean['cast'].fillna('').apply(lambda x: [i.strip() for i in x.split(',') if i.strip()])
cast_df = df_clean.explode('cast')

# Explode Director for power bi model
df_clean['director'] = df_clean['director'].fillna('').apply(lambda x: [i.strip() for i in x.split(',') if i.strip()])
director_df = df_clean.explode('director')

# Explode Country for power bi model
df_clean['country'] = df_clean['country'].fillna('').apply(lambda x: [i.strip() for i in x.split(',') if i.strip()])
country_df = df_clean.explode('country')

# Export each exploded dataset separately for Power BI
genre_df.to_excel("netflix_by_genre.xlsx", index=False)
cast_df.to_excel("netflix_by_cast.xlsx", index=False)
director_df.to_excel("netflix_by_director.xlsx", index=False)
country_df.to_excel("netflix_by_country.xlsx", index=False)